# Segundo EP MAC0209 - 2020
### Integrantes:
+ Eduardo Brancher Urenha 
+ Lourenço Henrique Moinheiro Martins Sborz Bogo
+ Miguel de Mello Carpi

#### Bibliotecas e funções úteis


In [ ]:
## Bibliotecas utilizadas

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
import random
import itertools
from time import time

from  matplotlib import animation, rc, rcParams
from IPython.display import HTML
import csv
#rcParams['animation.embed_limit'] = 2**7 ## É bom ter memória para isso
##Recomendamos não usar as animações no modo LOOP, pois muitas animações tocando em conjunto podem quebrar 
##o server do jupyter notebook.

In [ ]:
## Abstração para diminuir o trabalho necessário para plotar os dados
class dabs:

    def __init__(self, data = [], name='', style='--r'):
        self.data  = data
        self.name  = name
        self.style = style


In [ ]:
## Funções para plotar gráficos
def plota(vx, vy, title='', style='--',save=1):
    plt.plot(vx.data, vy.data, style)
    plt.xlabel(vx.name)
    plt.ylabel(vy.name)
    plt.title(vy.name+' x '+vx.name if len(title) == 0 else title)
    if save==1:
        plt.savefig(title+'.png')
    plt.show()



def plota_vetor(vetores, vt, xlabel='entrada', ylabel='saída', title='meu-gráfico'):

    def get(f, vetores):
        m = 0
        for v in vetores:
            m = f(m, f(v.data))
        return m

    plt.xlim(0, vt[-1])
    plt.ylim(get(min, vetores),  get(max, vetores))
    
    for v in vetores:
        plt.plot(vt, v.data, v.style, label=v.name)

      
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.legend()
    plt.savefig(title)
    plt.show()



    
def plt2(v1, v2, style1='--r', style2='--b', xlabel='tempo', 
         title='meu-grafico'):

    plt.plot([x for x in range(len(v1.data))], v1.data, style1, label=v1.name)
    plt.plot([x for x in range(len(v2.data))], v2.data, style2, label=v2.name)

    plt.xlabel(xlabel)
    plt.title(title)
    plt.legend()
    plt.show()


In [ ]:
## Funções para escrever os dados em um arquivo csv
def write_csv(dvector, name='meu-csv'): 
    '''
    Exemplo de uso:
    d1 = dabs([1, 2, 3, 4], name='d1')
    d2 = dabs([1, 4, 3, 8], name='d2')
    d3 = dabs([1, 2, 3, 18], name='')
    dv = [d1, d2, d3]
    write_csv(dv)
    '''
    if len(dvector) == 0:
        printf("write_csv: Bad input!")
        return 1
    
    col_names = [d.name for d in dvector]

    if not '.csv' in name:
        name += '.csv'
    
    with open(name, 'w', newline='') as fl:
        wrt = csv.writer(fl)
        wrt.writerow(col_names)
        for i in range(len(dvector[0].data)):
            wrt.writerow([x.data[i] for x in dvector])
    
    return 0

## Armazena os dados da simulação em um csv
def model2csv(model, name):
    name = name+'.csv'
    
    f = open(name, 'w', newline='')

    head = ['dado', 'ilha', 'alpha', 'n0', 't0', 'A', 'lambda'] + ['t = {0:.1f}'.format(x) for x in ilha.timeVector]
    wrt = csv.DictWriter(f, fieldnames=head)
    wrt.writeheader()

    ls = []
    ls.append(['infectados', model.name, model.alpha, model.pop, model.t0, model.A, model.lamb] + [x for x in model.infected])
    ls.append(['aumento', model.name, model.alpha, model.pop, model.t0, model.A, model.lamb] + [x for x in model.increases])

    for l in ls:
        wrt.writerow({k:v for (k, v) in zip(head, l)})

    f.close()

## Modelo dinâmico do COVID-19
$ \frac{dN}{dt} = \alpha \left(1 - \frac{N}{\eta t^2} \right)N - \left(\frac{2\lambda t^2 - 1}{t} - \frac{\lambda {t_0}^2}{t}e^{-\lambda(t- t_0)^2}\right)N$

In [ ]:
## Autores: Lourenço Henrique Moinheiro Martins Sborz Bogo, Miguel de Mello Carpi
## Co-autor: Eduardo Brancher Urenha 

## Classe representando o modelo
class DynamicModel:

    def __init__(self, alpha, n0, t0, lamb, A, name):
        self.alpha        = alpha     # Taxa de crescimento do vírus
        self.pop          = n0        # População infectada inicial
        self.t0           = t0        # Primeiro dia do isolamento
        self.lamb         = lamb      # Depende da taxa de replicação do vírus
        self.A            = A         # Permissividade do meio
        self.eta          = 2*A/t0    # Permissividade do meio + isolamento

        self.timeVector = []
        self.increases  = []
        self.infected   = []

        self.name = name

        self.simu = {'d0' : 0, 'nd' : 0, 'dt' : 0}


        
    def __str__(self):
        return self.name + f' alpha: {self.t0}' + f' ini infec: {self.pop}' +\
        f' ini qrt: {self.t0}' + f' A: {self.A}' + f' eta: {self.eta}'


    def __c1(self, t, N):
        return 1-(N/(self.eta*t*t))


    def __c2(self, t):
        return (2*self.lamb*t*t-1)/t-((self.lamb*self.t0*self.t0)/t)*\
            (np.exp(-self.lamb*(t-self.t0)*(t-self.t0)))


    def __dN(self, N, t):
        return self.__c1(t, N)*self.alpha*N - self.__c2(t)*N

    def correctAlpha(self):
        '''
        Função para corrigir o valor de alpha. Segundo o artigo do Sonnino, 
        alpha é um parâmetro que depende de t0 e de lambda
        '''
        self.alpha = 2*self.t0*self.lamb
        
         
    def simulate(self, d0, nd, dt=0.1):
        '''
        Usando os parametros definidos na construção do modelo, simula esse 
        modelo dinâmico a partir do dia 'd0'.
        '''
        self.timeVector  = []
        self.infected    = []
        self.increases   = []

        self.simu['d0'] = d0
        self.simu['nd'] = nd
        self.simu['dt'] = dt

        
        n  = self.pop
        inc = self.pop
        for t in np.arange(d0, nd, dt):
            self.timeVector.append(t)
            self.infected.append(n)
            self.increases.append(inc)
            inc = self.__dN(n, t+dt)
            n += inc*dt
            

In [ ]:
## Simulação da Itália 
ilha = DynamicModel(alpha=0.2,        # Taxa de crescimento do vírus
                        n0=324,       # População infectada inicialmente(obtido a partir do site da OMS considerando o gráfico do Sonnino)
                        t0=70.6,      # Início do isolamento
                        A=2135,       # Permissividade do meio
                        lamb=0.0014,  # Varíavel atrelada a taxa de replicação do vírus
                        name="Italia")

ilha.simulate(d0=25, nd=80)

In [ ]:
## Italia
tt = ilha.name+f'-t0={ilha.t0}'
dtempo = dabs(ilha.timeVector, name='Tempo (dias)')
dinfected = dabs(ilha.infected, name='Número de Infectados')

plota(dtempo, dinfected,title=tt, style='--g')
model2csv(ilha,tt)

### Primeira parte: Simulação em uma ilha isolada

In [ ]:
seed = 'cknorris'
random.seed(seed)
alpha  = random.uniform(0.2, 0.4)
n0     = 1
t0     = random.randint(10, 100)
t0_low = t0//5
A      = random.randint(360,14000)
lamb   = random.uniform(0.001, 0.003)

In [ ]:
## Simulação de uma única ilha 
ilha = DynamicModel(alpha=alpha,   # Taxa de crescimento do vírus
                        n0=n0,     # População infectada inicialmente
                        t0=t0,     # Início do isolamento
                        A=A,       # Permissividade do meio
                        lamb=lamb, # Varíavel atrelada a taxa de replicação do vírus
                        name=seed)

ilha.simulate(d0=1, nd=200)

In [ ]:
## Usando valores aleatórios
tt = ilha.name+f'-t0={ilha.t0}'
dtempo = dabs(ilha.timeVector, name='Tempo (dias)')
dinfected = dabs(ilha.infected, name='Número de Infectados')

plota(dtempo, dinfected,title=tt, style='--g')
model2csv(ilha,tt)

In [ ]:
## Agora usando t0_low
ilha.t0 = t0_low
ilha.simulate(1, 200)

## Gráfico para uma ilha usando t0_low
tt = ilha.name+f'-t0={ilha.t0}'
dtempo = dabs(ilha.timeVector, name='Tempo (dias)')
dinfected = dabs(ilha.infected, name='Número de Infectados')

plota(dtempo, dinfected,title=tt, style='--g')
model2csv(ilha,tt)

In [ ]:
## Agora usando alpha corrigido
ilha.t0 = t0
ilha.correctAlpha()
ilha.simulate(1, 200)
## Gráfico para uma ilha usando alpha corrigido
tt=ilha.name+f'-t0={ilha.t0}-alpha-corrigido'
dtempo = dabs(ilha.timeVector, name='Tempo (dias)')
dinfected = dabs(ilha.infected, name='Número de Infectados')

plota(dtempo, dinfected,title=tt, style='--g')
model2csv(ilha, tt)

In [ ]:
## Usando t0_low e alpha corrigido
ilha.t0 = t0_low
ilha.correctAlpha()
ilha.simulate(1, 200)
## Gráfico para uma ilha usando t0_low e alpha corrigido
tt=ilha.name+f'-t0={ilha.t0}-alpha-corrigido'
dtempo = dabs(ilha.timeVector, name='Tempo (dias)')
dinfected = dabs(ilha.infected, name='Número de Infectados')

plota(dtempo, dinfected,title=tt, style='--g')
model2csv(ilha, tt)

In [ ]:
## Animação para uma ilha
fig, ax = plt.subplots(1, figsize=(10, 10))

ax.set_xlabel('Tempo (dias)')
ax.set_ylabel('Infectados')
ax.set_xlim(0, ilha.simu['nd']*1.1)
ax.set_ylim(min(dinfected.data), max(dinfected.data))

l, = ax.plot([], [], dinfected.style, lw=2, label=dinfected.name)


def up(i):  
  l.set_data(dtempo.data[:i*10], dinfected.data[:i*10])
  return l,


animacao_ilha = animation.FuncAnimation(fig, 
                                         up, 
                                         blit=True, 
                                         interval=20, 
                                         frames=range(ilha.simu['nd']))

plt.title('Simulação para uma ilha isolada')
plt.legend()
rc('animation', html='jshtml')
animacao_ilha

### Segunda parte: Simulação em cinco ilhas isoladas

In [ ]:
## Simulação randomizada não respeita as dependências dos parâmetros do modelo

MAX_SIMU = 100

rndf = random.uniform
rndi = random.randint

ilhas = [DynamicModel(alpha=rndf(0.1, 0.4), 
                      n0=1, 
                      t0=rndi(1, MAX_SIMU), 
                      A=rndi(360,14000), 
                      lamb=rndf(0.005, 0.01), 
                      name=f'Ilha: {i}') for i in range(5)]

v = []
style = ['--r', '--b', '--g', '--k', '--y']
for i, ilha in enumerate(ilhas):
  ilha.simulate(0, MAX_SIMU)
  v.append(dabs(ilha.infected, ilha.name, style[i]))


In [ ]:
## CSV para a simulação das cinco ilhas

name = '5ilhas_isoladas.csv'
    
f = open(name, 'w', newline='')

head = ['dado','ilhas', 'alpha', 'n0', 't0', 'A', 'lambda'] + ['t = {0:.1f}'.format(x) for x in np.arange(0, MAX_SIMU, 0.1)]
wrt = csv.DictWriter(f, fieldnames=head)
wrt.writeheader()

ls = []
for i in ilhas:
    ls.append(['infectados',i.name, i.alpha, i.pop, i.t0, i.A, i.lamb] + [x for x in i.infected])
    ls.append(['aumento', i.name, i.alpha, i.pop, i.t0, i.A, i.lamb] + [x for x in i.increases])
    
    
for l in ls:
        wrt.writerow({k:v for (k, v) in zip(head, l)})

f.close()

In [ ]:
df = pd.read_csv(name)
df

In [ ]:
## Gráfico das cinco ilhas
plota_vetor(v, np.arange(0, MAX_SIMU, 0.1), 'Tempo', 'Infectados', 'grafico_5_ilhas')

In [ ]:
## Animação das cinco ilhas

fig, ax = plt.subplots(1, figsize=(10, 10))

tvec  = np.arange(0, MAX_SIMU, 0.1)
lines = []

def get(f, vs):
    m = 0
    for v in vs:
        m = f(m, f(v.data))
    return m

ax.set_xlabel('Tempo (dias)')
ax.set_ylabel('Infectados')
ax.set_xlim(0, MAX_SIMU*1.1)
ax.set_ylim(get(min, v), get(max, v))
for d in v:
    l, = ax.plot([], [], d.style, lw=2, label=d.name)
    lines.append(l)

def upd(i):
  for j, line in enumerate(lines):
    line.set_data(tvec[:i*10], v[j].data[:i*10])
  return tuple(lines)




animacao_ilhas = animation.FuncAnimation(fig, 
                                         upd, 
                                         blit=True, 
                                         interval=20, 
                                         frames=range(MAX_SIMU))
plt.title('Simulação para cinco ilhas isoladas')
plt.legend()
rc('animation', html='jshtml')
animacao_ilhas

In [ ]:
## Gráfico da norma do vetor de estados
def norm(x, i):
    sum = 0
    for element in x:
        sum += element.infected[i]**2
    return np.sqrt(sum)


dtempo = dabs(np.arange(0, MAX_SIMU, 0.1), name='Tempo(dias)')
dnorma = dabs([norm(ilhas, i) for i in range(MAX_SIMU*10)], name='Norma', style='--g')

plota(dtempo, dnorma, title='Grafico-da-norma')

In [ ]:
## Animação da norma do vetor de estados

fig, ax = plt.subplots(1, figsize=(10, 10))

ax.set_xlabel('Tempo (dias)')
ax.set_ylabel('Norma')
ax.set_xlim(0, MAX_SIMU*1.1)
ax.set_ylim(0, max(dnorma.data)*1.1)

l, = ax.plot([], [], '--g', lw=2, label='Norma')

#print(dnorma.data)

def up(i):  
  l.set_data(dtempo.data[:i*10], dnorma.data[:i*10])
  return l,


animacao_norma = animation.FuncAnimation(fig, 
                                         up, 
                                         blit=True, 
                                         interval=20, 
                                         frames=range(MAX_SIMU))


plt.title('Norma do vetor')
plt.legend()
rc('animation', html='jshtml')
animacao_norma


In [ ]:
## Gráfico comparativo entre as ilhas
dilhas = [dabs(x.infected, x.name) for x in ilhas]

for i, j in itertools.combinations(dilhas, 2):
    plota(i, j,save=0)
    plota(j, i,save=0)


In [ ]:
## Célula para animar o gráfico da ilha_i X ilha_j, i != j e 0 <= i, j < 5 

i = 0
j = 1
ilha_i = dabs(ilhas[i].infected, ilhas[i].name)
ilha_j = dabs(ilhas[j].infected, ilhas[j].name)

fig, ax = plt.subplots(1, figsize=(10, 10))

ax.set_xlabel(ilha_i.name)
ax.set_ylabel(ilha_j.name)
ax.set_xlim(0, max(ilha_i.data)*1.1)
ax.set_ylim(0, max(ilha_j.data)*1.1)

l, = ax.plot([], [], '--g', lw=2, label=ilha_j.name+" x "+ilha_i.name)


def up(i):  
  l.set_data(ilha_i.data[:i*10], ilha_j.data[:i*10])
  return l,


animacao_ilha2 = animation.FuncAnimation(fig, 
                                         up, 
                                         blit=True, 
                                         interval=20, 
                                         frames=range(ilha.simu['nd']))

plt.title(f'Simulação para ilha_{j} X ilha_{i}')
plt.legend()
rc('animation', html='jshtml')
animacao_ilha2

## Modelo estocástico do COVID-19

In [ ]:
##Autor: Eduardo Brancher Urenha
##Co-autores: Miguel de Mello Carpi, Lourenço Henrique Moinheiro Martins Sborz Bogo

import random
from time import time

seed = "XARABACAIA"
random.seed(seed)

## Modos do programa
Clustering = 1
Sick_quarantined = 1
Days_limit = 600
## Contadores
Infected_total = 0
Max_infected = 0
Immune_total = 0
Dead_total = 0
R_value = 0
Daily_new_cases = 0

## Numero de População
Total_pop = 10000
Living_pop = Total_pop
Uninfected = Living_pop
Starting_infected = 1 #menor possivel pra nao explodir

## Parametros da doença
Death_rate = 0.001 #taxa de morte DIARIA
Incubation_time = 6
Illness_course = Incubation_time + 21 #duração
Infection_chance_upon_meeting = 0.12
Minimum_incubation_for_infection = 4

## Dinamica Social
Seed_clustersize = 6 #cluster normal: 0.001
# Uso de clustersize: 
# Seed_clustersize é o valor do inner_cluster. 
# middle_cluster tem tamanho inner_cluster*Middle_factor
# e outer tem tamanho inner_cluster*Outer_factor
Quarantine_chance = 0.50
Max_meetings = 10 #por dia
Middle_factor = 5
Outer_factor = 20
Inner_cluster_chance = 0.90
Middle_cluster_chance = 0.90
Outer_cluster_chance = 0.90 #o minimo num cluster tem que ser 50%, do contrario, será mais facil encontrar alguem da pop em geral.

## Não Mexer
Population = []

class Person:
    
    global Total_pop
    global Population
    global Max_meetings
    global Quarantine_chance
    
    def __init__(self, i):
        self.id = i
        self.quarantine_chance = Quarantine_chance
        self.max_meetings = random.randint(1, Max_meetings)
        self.inner_cluster = []
        self.inner_cluster_chance = Inner_cluster_chance
        self.middle_cluster = []
        self.middle_cluster_chance = Middle_cluster_chance
        self.outer_cluster = []
        self.outer_cluster_chance = Outer_cluster_chance
        self.inner_clustersize = 0
        self.middle_clustersize = 0
        self.outer_clustersize = 0
        self.quarantine = 0
        self.dead = 0
        self.infected = 0
        self.pending_infection = 0
        self.infections_caused = 0
        self.infection_time = 0
        self.immune = 0
        self.meet_chance = random.random()
        
    def __str__(self):
        string = ("ID: " + str(format(self.id, '7d')) + " meet chance: " + str(format(self.meet_chance, '.4f')) + " inner cluster: ")
        for i in self.inner_cluster:
            string = string + str(i)
            string = string + " "
        string = string + "inner clustersize: " + str(self.inner_clustersize)
        string = string + (" middle cluster: ")
        for i in self.middle_cluster:
            string = string + str(i)
            string = string + " "
        string = string + "middle clustersize: " + str(self.middle_clustersize)
        string = string + (" outer cluster: ")
        for i in self.outer_cluster:
            string = string + str(i)
            string = string + " "
        string = string + "outer clustersize: " + str(self.outer_clustersize)
        string = string + ("\nImmune: " + str(self.immune) + " infected: " + str(self.infected) + " infection days: " + str(self.infection_time) + " dead: " + str(self.dead))
        return string
    
        
     
    def is_in_Cluster(self, num, cluster):
        belongs_to_cluster = 0
        for person_id in cluster:
            if num == person_id:
                belongs_to_cluster = 1
        return belongs_to_cluster
    
    def generateClusters(self):
        
        global Seed_clustersize
        global Middle_factor
        global Outer_factor
        
        inner_clustersize = random.randint(0, Seed_clustersize)
        if inner_clustersize == 0:
            self.inner_cluster_chance = 0
        middle_clustersize = random.randint(0, Seed_clustersize*Middle_factor)
        if middle_clustersize == 0:
            self.middle_cluster_chance = 0
        outer_clustersize = random.randint(0, Seed_clustersize*Outer_factor)
        if outer_clustersize == 0:
            self.outer_cluster_chance = 0
            
        for i in range(0, inner_clustersize):
            num = random.randint(0, Total_pop - 1) 
            while (num == self.id):
                num = random.randint(0,Total_pop - 1)
            if self.is_in_Cluster(num, self.inner_cluster) == 0:
                self.inner_cluster.append(num)
                self.inner_clustersize = len(self.inner_cluster)
                Population[num].inner_cluster.append(self.id)
                Population[num].inner_clustersize = len(Population[num].inner_cluster)
            
        for i in range (0, middle_clustersize):
            num = random.randint(0, Total_pop - 1) 
            while (num == self.id):
                num = random.randint(0,Total_pop - 1)
            if self.is_in_Cluster(num, self.middle_cluster) == 0 and self.is_in_Cluster(num, self.inner_cluster) == 0:
                self.middle_cluster.append(num)
                self.middle_clustersize = len(self.middle_cluster)
                Population[num].middle_cluster.append(self.id)
                Population[num].middle_clustersize = len(Population[num].middle_cluster)
        
        for i in range(0, outer_clustersize):
            num = random.randint(0, Total_pop - 1) 
            while (num == self.id):
                num = random.randint(0,Total_pop - 1)
            if self.is_in_Cluster(num, self.middle_cluster) == 0 and self.is_in_Cluster(num, self.inner_cluster) == 0 and self.is_in_Cluster(num, self.outer_cluster) == 0:
                self.outer_cluster.append(num)
                self.outer_clustersize = len(self.outer_cluster)
                Population[num].outer_cluster.append(self.id)
                Population[num].outer_clustersize = len(Population[num].outer_cluster)
        
def processMeeting(person1, person2):
    
    global Infected_total
    global Uninfected
    global Max_infected
    global Infection_chance_upon_meeting
    global Daily_new_cases
    
    if person1.dead == 1 or person2.dead == 1:
        return
    
    if person1.quarantine == 1 or person2.quarantine == 1:
        return
    
    infection_roll = random.random()
    if infection_roll >= Infection_chance_upon_meeting:
        return
    
    if person1.infected == 1 and person2.infected == 0:
        if person2.immune == 0 and person1.infection_time > Minimum_incubation_for_infection:
            person2.pending_infection = 1
            person1.infections_caused = person1.infections_caused + 1

            
    elif (person2.infected == 1 and person1.infected == 0):
        if person1.immune == 0 and person2.infection_time > Minimum_incubation_for_infection:
            person1.pending_infection = 1
            person2.infections_caused = person2.infections_caused + 1
            
            
def activateInfection(person):
    
    global Infected_total
    global Uninfected
    global Max_infected
    global Daily_new_cases
    
    person.pending_infection = 0
    person.infected = 1
    Infected_total = Infected_total + 1
    Uninfected = Uninfected - 1
    Max_infected = Max_infected + 1
    Daily_new_cases = Daily_new_cases + 1
            
def processIllness(person):
        
        global Infected_total
        global Immune_total
        global Dead_total
        global Living_pop
        global Death_rate
        global Sick_quarantined
        global Incubation_time
        global Illness_course
    
        if person.infected == 1:
            person.infection_time = person.infection_time + 1 #correto
    
        
        if Sick_quarantined == 1 and person.infection_time > Incubation_time:
            quarantine = random.random()
            if quarantine < person.quarantine_chance:
                person.quarantine = 1
        
        if person.infected == 1 and person.infection_time > Incubation_time:
            death = random.random()
            if death < Death_rate:
                Dead_total = Dead_total + 1
                Infected_total = Infected_total - 1
                Living_pop = Living_pop - 1
                person.dead = 1 #RIP
                person.infected = 0
                person.quarantine = 0
            
        if person.infected == 1 and person.infection_time == Illness_course and person.dead == 0:
            person.infected = 0 #here lies PRESON
            person.immune = 1
            person.quarantine = 0
            Infected_total = Infected_total - 1
            Immune_total = Immune_total + 1 #OK
                

def updatePopulation():
    
    global Population
    global Inner_cluster_chance
    global Middle_cluster_chance
    global Outer_cluster_chance
    global Clustering
    global Total_pop
    
        
    
    for person in Population:
        if person.pending_infection == 1:
            activateInfection(person)
        if person.infected == 1:
            processIllness(person)
        
        if person.dead: #pessoa está morta
            continue
                   
        #encontros
        #if person.quarantine == 1:
            #continue
            
        #print(person)    
        meeting_roll = random.random()        
        if meeting_roll < person.meet_chance:
            meeting_num = random.randint(1, person.max_meetings)
            
            for i in range(0, meeting_num):
                if Clustering == 1:
                    if person.inner_clustersize > 0:
                        cluster_roll = random.random()          
                        if cluster_roll < person.inner_cluster_chance:
                            other = random.randint(0, person.inner_clustersize-1)
                            #print(other)
                            processMeeting(person, Population[person.inner_cluster[other]])
                        elif person.middle_clustersize > 0:
                            cluster_roll = random.random()
                            if cluster_roll < person.middle_cluster_chance:
                                other = random.randint(0, person.middle_clustersize-1)
                                #print(other)
                                processMeeting(person, Population[person.middle_cluster[other]])
                            elif person.outer_clustersize > 0:
                                cluster_roll = random.random()
                                if cluster_roll < person.outer_cluster_chance:
                                    other = random.randint(0, person.outer_clustersize-1)
                                    #print(other)
                                    processMeeting(person, Population[person.outer_cluster[other]])
                                else:
                                    other = random.randint(0, Total_pop-1)
                                    while Population[other].id == person.id:
                                        other = random.randint(0, Total_pop-1)
                                    processMeeting(person, Population[other]) #ok
                else:
                    other = random.randint(0, Total_pop-1)
                    while Population[other].id == person.id:
                        other = random.randint(0, Total_pop-1)
                    processMeeting(person, Population[other]) #ok

                        
def main():

    global Infected_total
    global Population
    global Death_rate
    global Incubation_time
    global Illness_course
    global Clustering
    global Seed_clustersize
    global Days_limit
    global Sick_quarantined 
    global Quarantine_chance
    global Infected_total
    global Immune_total
    global Dead_total
    global Max_meetings
    global Total_pop
    global Living_pop
    global Max_infected
    global Uninfected
    global Starting_infected
    global R_value
    global Daily_new_cases
    global Middle_factor
    global Outer_factor
    global Inner_cluster_chance
    global Middle_cluster_chance
    global Outer_cluster_chance
    
    
    cronometro2 = time()
    cronometro = time()
    
    days_active = 0
    days_count = 0
    infected_array = []
    R_array = []
    immune_array = []
    dead_array = []
    herd_immunity_array = []
    max_infected_array = []
    max_infected_proportion_array = []
    uninfected_proportion_array = []
    daily_new_array = []
    i = 0
    
    while i < Total_pop:
        new_individual = Person(i)
        Population.append(new_individual)
        i = i + 1
    
    i = 0
    while i < Total_pop:
        Population[i].generateClusters()
        i = i + 1
    
    print("CLUSTERS GERADOS")
    print("Tempo pra gerar os clusters: ", time() - cronometro2)
    
    #for person in Population:
        #person.inner_cluster.sort()
        #person.middle_cluster.sort()
        #person.outer_cluster.sort()
    
    for i in range(0, Starting_infected):
        rand = random.randint(0, Total_pop - 1)
        if (Population[rand].infected == 0):
            Population[rand].infected = 1
            Population[rand].quarantine_chance = 0 #precisa ser 0 porque pode ser que nao role simulação alguma se nao for
            Infected_total = Infected_total + 1 #ok
            Max_infected = Max_infected + 1
            Uninfected = Uninfected - 1
    
    max_R_value = 0
    
    while days_count < Days_limit:
        current_R_value = 0
        previous_infected = Max_infected
        Daily_new_cases = 0
        
        herd_immunity = Immune_total / Living_pop
        current_infected_proportion = Infected_total / Living_pop
        max_infected_proportion = Max_infected / Total_pop
        uninfected_proportion = Uninfected / Total_pop #ok
        
        infected_array.append(Infected_total)
        immune_array.append(Immune_total)
        dead_array.append(Dead_total)
        max_infected_array.append(Max_infected)
        
        herd_immunity_array.append(herd_immunity)
        uninfected_proportion_array.append(uninfected_proportion)
        
        updatePopulation()
        
        daily_new_array.append(Daily_new_cases)
        
        
        
        for person in Population:
            if (person.infected == 1):
                current_R_value = current_R_value + person.infections_caused
            
        current_R_value = current_R_value / previous_infected
        R_array.append(current_R_value)
       
        if Infected_total > 0:
            days_active = days_active + 1
                      
        days_count = days_count + 1
        #for i in range(0, 10):
           #print(population[i])
        
        R_value = R_value + current_R_value
    

        
    R_value = R_value / days_active
    
    #print(herd_immunity_array)
    print("Total de infectados: ")
    print(infected_array)
    #print(uninfected_proportion_array)
    #print(R_array)
    #print(daily_new_array)
    print("Herd Immunity: ", herd_immunity_array[-1])
    print("Current infected: ", infected_array[-1])
    print("Maximum infected simultaneously: ", max(infected_array))
    print("Uninfected proportion: ", uninfected_proportion_array[-1])
    print("R-Value: ", R_value)
    print("Max-R: ", max(R_array))
    
    infectados = dabs(infected_array, "Infectados", "--r")
    tempo = dabs(range(0, len(infected_array)), "Dias", "--r")
    plota(tempo, infectados, "Infectados x Tempo", "--r")
    
    average_inner_cluster = 0 
    average_middle_cluster = 0
    average_outer_cluster = 0
    
    for person in Population:
        average_inner_cluster = average_inner_cluster + person.inner_clustersize
        average_middle_cluster = average_middle_cluster + person.middle_clustersize
        average_outer_cluster = average_outer_cluster + person.outer_clustersize
    
    average_inner_cluster = average_inner_cluster / Total_pop
    average_middle_cluster = average_middle_cluster / Total_pop
    average_outer_cluster = average_outer_cluster / Total_pop
    print("Average inner cluster size: ", average_inner_cluster)
    print("Average middle cluster size: ", average_middle_cluster)
    print("Average outer cluster size: ", average_outer_cluster)
        
    print("Total time:", time() - cronometro)
    
    d0 = [i for i in range (0, Days_limit)]
    d0 = dabs(d0, "Dias")
    d1 = dabs(infected_array, "Numero de infectados")
    d2 = dabs(daily_new_array, "Casos novos nesse dia")
    d3 = dabs(dead_array, "Mortes")
    d4 = dabs(immune_array, "Imunes(curados)")
    d5 = dabs(herd_immunity_array, "Imunidade de manada")
    d6 = dabs(max_infected_array, "Proporcao de contagio")
    d7 = dabs(uninfected_proportion_array, "Proporcao de nao infectados")
    d8 = dabs(R_array, "Valores de R")
    d9 = ["Clustering", "Sick_quarantined", "Days_limit", "Total_pop", "Starting_infected", "Death_rate", "Incubation_time", "Illness_course", 
          "Infection_chance_upon_meeting","Quarantine_chance", "Minimum_incubation_for_infection", "Seed_clustersize", "Max_meetings", "Middle_factor", "Outer_factor",
          "Inner_cluster_chance", "Middle_cluster_chance", "Outer_cluster_chance"]
    d10 = [Clustering, Sick_quarantined, Days_limit, Total_pop, Starting_infected, Death_rate, Incubation_time, Illness_course, 
          Infection_chance_upon_meeting, Quarantine_chance, Minimum_incubation_for_infection, Seed_clustersize, Max_meetings, Middle_factor, Outer_factor,
          Inner_cluster_chance, Middle_cluster_chance, Outer_cluster_chance]
    for i in range(18, Days_limit):
        d9.append("")
        d10.append("")
    d9 = dabs(d9, "Param_name")
    d10 = dabs(d10, "Params")
    
    dv = [d0, d1, d2, d3, d4, d5, d6, d7, d8, d9, d10]
    
    write_csv(dv, seed)
    
    
    #-----END OF MAIN FUNCTION-----#
    
main()


#for i in population:
   #print(i)

print("Pop. total: ", Total_pop)
print("Total de imunes: ", Immune_total)
print("Total de mortos: ", Dead_total)
print("Pessoas vivas: ", Living_pop)

#for person in Population:
#    if person.immune and person.dead:
#        print("DEAD MAN WALKING")
#        print(person)
#    if person.immune and person.infected:
#        print ("DOUBLE INFECTION")
#        print(person)
#    if (person.infected and person.dead):
#        print ("ZOMBIE")
#        print(person)
    
print("end")
